In [27]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
import numpy as np
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import one_hot, Tokenizer
from keras.utils import pad_sequences
from keras.models import Sequential
import tensorflow_hub as hub
import tensorflow_text as text
from keras.layers import Embedding,Conv1D,GlobalMaxPooling1D,MaxPooling1D,Dense,AdditiveAttention,MultiHeadAttention,Bidirectional,LSTM,Flatten,Cropping3D,UpSampling3D,Attention,Dropout
import tensorflow as tf

In [28]:
# importing data
data=pd.read_csv('/kaggle/input/stockmarket-sentiment-dataset/stock_data.csv')

In [29]:
#checking for null data
print(data['Text'].isnull().sum())
print(data['Sentiment'].isnull().sum())
data

0
0


,Text,Sentiment
0,Kickers on my watchlist XIDE TIT SOQ PNK CPW B...,1
1,user: AAP MOVIE. 55% return for the FEA/GEED i...,1
2,user I'd be afraid to short AMZN - they are lo...,1
3,MNTA Over 12.00,1
4,OI Over 21.37,1
...,...,...
5786,Industry body CII said #discoms are likely to ...,-1
5787,"#Gold prices slip below Rs 46,000 as #investor...",-1
5788,Workers at Bajaj Auto have agreed to a 10% wag...,1
5789,"#Sharemarket LIVE: Sensex off day’s high, up 6...",1


In [30]:
data.isnull().values.any()

False

In [31]:
data['Text'][1]

'user: AAP MOVIE. 55% return for the FEA/GEED indicator just 15 trades for the year.  AWESOME.  '

In [32]:
TAG_RE = re.compile(r'<[^>]+>')

def remove_tags(text):
    '''Removes HTML tags: replaces anything between opening and closing <> with empty space'''

    return TAG_RE.sub('', text)

In [33]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [34]:
def preprocess_text(sen):
    '''Cleans text data up, leaving only 2 or more char long non-stepwords composed of A-Z & a-z only
    in lowercase'''
    
    sentence = sen.lower()

    # Remove html tags
    sentence = remove_tags(sentence)

    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sentence)

    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)  # When we remove apostrophe from the word "Mark's", the apostrophe is replaced by an empty space. Hence, we are left with single character "s" that we are removing here.

    # Remove multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)  # Next, we remove all the single characters and replace it by a space which creates multiple spaces in our text. Finally, we remove the multiple spaces from our text as well.

    # Remove Stopwords
    pattern = re.compile(r'\b(' + r'|'.join(stopwords.words('english')) + r')\b\s*')
    sentence = pattern.sub('', sentence)

    return sentence

In [35]:
X = []
sentences = list(data['Text'])
for sen in sentences:
    X.append(preprocess_text(sen))

In [36]:
print(X[0])
print(data['Text'][0])

kickers watchlist xide tit soq pnk cpw bpz aj trade method method see prev posts
Kickers on my watchlist XIDE TIT SOQ PNK CPW BPZ AJ  trade method 1 or method 2, see prev posts


In [37]:
data['Sentiment']=data['Sentiment'].map({-1:0,1:1})

In [38]:
# x=np.array(X)
y=np.array(data['Sentiment'])
x=X

In [39]:
x_train,x_test,y_train,y_test=train_test_split(x,y)
y_train.shape,y_test.shape

((4343,), (1448,))

In [40]:
y_train=np.array(y_train)
y_test=np.array(y_test)
x_train=np.array(x_train)
x_test=np.array(x_test)

In [41]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-768_A-12/2",trainable=True)

In [42]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)
# outputs=Reshape((256,1))(outputs["pooled_output"])

# CNN layer
cnn_layer = Conv1D(64,3,padding="same", activation='relu')(outputs["sequence_output"])
cnn_layer=Dropout(0.2)(cnn_layer)
cnn_layer = MaxPooling1D()(cnn_layer)

# CNN layer
cnn_layer2 = Conv1D(128, 5,padding="same",activation='relu')(cnn_layer)
# cnn_layer2 = MaxPooling1D()(cnn_layer2)
cnn_layer2=Dropout(0.2)(cnn_layer2)
#CNN layer
# num_filters = 256
# kernel_size = 7
# cnn_layer2 = Conv1D(filters=num_filters, kernel_size=kernel_size, activation='relu')(cnn_layer2)
# cnn_layer2=Dropout(0.2)(cnn_layer2)
# cnn_layer2 = MaxPooling1D()(cnn_layer2)
# LSTM layer

lstm_layer = Bidirectional(LSTM(64, return_sequences=True))(cnn_layer2)
lstm_layer = Bidirectional(LSTM(128))(lstm_layer)
#attention_layer=attention()(lstm_layer)
# lstm_layer = Attention()([lstm_layer, lstm_layer])
# lstm_layer=Flatten()(lstm_layer)

# Neural network layers
# l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(lstm_layer)


# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [43]:
# model.add(Embedding(vocab_length,100,weights=[embedding_matrix], input_length=maxlen))
# model.add(Conv1D(128,5,padding ="same",activation="relu"))
# model.add(MaxPooling1D())
# model.add(Dropout(0.2))
# model.add(Conv1D(256,5,padding="same",activation="relu"))
# #model.add(MultiHeadAttention(key_dim=128, value_dim=128, num_heads=1,value=10))
# model.add(MaxPooling1D())
# model.add(Dropout(0.2))
# #model.add(Bidirectional(LSTM(20)))
# model.add(Bidirectional(LSTM(40)))
# #model.add(Flatten())
# # model.add(Cropping3D(cropping=((1, 1), (1, 1), (1, 1))))
# # model.add(UpSampling3D(size=(2, 2, 2)))
# # model.add(AdditiveAttention()) 
# #model.add(MultiHeadAttention(num_heads=2, key_dim=128, value=128))
# # model.add(Flatten())

# model.add(Dense(units=1,activation="sigmoid"))

In [44]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer_2 (KerasLayer)     {'input_type_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_word_ids':                                                
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128)}                                                    

In [45]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [46]:
# model.fit(x_train[:2000],y_train[:2000],epochs=10,batch_size=256)

In [47]:
model.fit(x_train,y_train,epochs=10,batch_size=128,validation_data=(x_test,y_test))

Epoch 1/10
17/17 [==============================] - 90s 4s/step - loss: 0.6630 - accuracy: 0.6346 - val_loss: 0.7185 - val_accuracy: 0.6202
Epoch 2/10
17/17 [==============================] - 74s 4s/step - loss: 0.6577 - accuracy: 0.6417 - val_loss: 0.6647 - val_accuracy: 0.6202
Epoch 3/10
17/17 [==============================] - 75s 5s/step - loss: 0.6534 - accuracy: 0.6417 - val_loss: 0.6579 - val_accuracy: 0.6202
Epoch 4/10
17/17 [==============================] - 77s 5s/step - loss: 0.6149 - accuracy: 0.6498 - val_loss: 0.5796 - val_accuracy: 0.7141
Epoch 5/10
17/17 [==============================] - 75s 5s/step - loss: 0.5550 - accuracy: 0.7329 - val_loss: 0.6270 - val_accuracy: 0.6754
Epoch 6/10
17/17 [==============================] - 74s 4s/step - loss: 0.5279 - accuracy: 0.7486 - val_loss: 0.5701 - val_accuracy: 0.7452
Epoch 7/10
17/17 [==============================] - 75s 5s/step - loss: 0.4531 - accuracy: 0.8073 - val_loss: 0.5937 - val_accuracy: 0.7376
Epoch 8/10
17/17 [==

In [48]:
model.fit(x_train,y_train,epochs=10,batch_size=256,validation_data=(x_test,y_test))

Epoch 1/10
17/17 [==============================] - 75s 4s/step - loss: 0.4163 - accuracy: 0.8264 - val_loss: 0.5575 - val_accuracy: 0.7210
Epoch 2/10
17/17 [==============================] - 74s 4s/step - loss: 0.3809 - accuracy: 0.8416 - val_loss: 0.6170 - val_accuracy: 0.7320
Epoch 3/10
17/17 [==============================] - 75s 4s/step - loss: 0.3355 - accuracy: 0.8692 - val_loss: 0.6498 - val_accuracy: 0.7189
Epoch 4/10
17/17 [==============================] - 75s 4s/step - loss: 0.3084 - accuracy: 0.8823 - val_loss: 0.6217 - val_accuracy: 0.7452
Epoch 5/10
17/17 [==============================] - 70s 4s/step - loss: 0.3228 - accuracy: 0.8736 - val_loss: 0.5653 - val_accuracy: 0.7106
Epoch 6/10
17/17 [==============================] - 75s 4s/step - loss: 0.3008 - accuracy: 0.8888 - val_loss: 0.6030 - val_accuracy: 0.7231
Epoch 7/10
17/17 [==============================] - 75s 4s/step - loss: 0.2683 - accuracy: 0.9044 - val_loss: 0.6726 - val_accuracy: 0.7465
Epoch 8/10
17/17 [==

In [49]:
# mofit(x_train[2000:],y_train[2000:],epochs=15,batch_size=280,validation_data=(x_test,y_test))

In [53]:
model.fit(x_train,y_train,epochs=5,batch_size=256,validation_data=(x_test,y_test))

Epoch 1/5
17/17 [==============================] - 75s 4s/step - loss: 0.1559 - accuracy: 0.9512 - val_loss: 0.8155 - val_accuracy: 0.7535
Epoch 2/5
17/17 [==============================] - 75s 4s/step - loss: 0.1327 - accuracy: 0.9606 - val_loss: 0.8196 - val_accuracy: 0.7555
Epoch 3/5
17/17 [==============================] - 75s 4s/step - loss: 0.1116 - accuracy: 0.9685 - val_loss: 0.9096 - val_accuracy: 0.7535
Epoch 4/5
17/17 [==============================] - 68s 4s/step - loss: 0.0944 - accuracy: 0.9744 - val_loss: 0.9647 - val_accuracy: 0.7459
Epoch 5/5
17/17 [==============================] - 73s 4s/step - loss: 0.1016 - accuracy: 0.9714 - val_loss: 0.9769 - val_accuracy: 0.7521


In [50]:
# prediction=model.predict(x_test)
y_test_pred=model.predict(x_test)
model.evaluate(x_test,y_test)



46/46 [==============================] - 5s 112ms/step - loss: 0.7401 - accuracy: 0.7521


[0.7400567531585693, 0.7520717978477478]

In [51]:
y_test_pred
threshold = 0.5

# Convert the data to 1 and 0 using the threshold
binary_data = (y_test_pred >= threshold).astype(int)

print(binary_data)
print(y_test_pred)
y_test_pred=binary_data

[[1]
 [1]
 [0]
 ...
 [0]
 [0]
 [0]]
[[0.9824576 ]
 [0.98093504]
 [0.04422473]
 ...
 [0.1247251 ]
 [0.05117709]
 [0.04734604]]


In [52]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_test_pred))


              precision    recall  f1-score   support

           0       0.68      0.65      0.67       550
           1       0.79      0.81      0.80       898

    accuracy                           0.75      1448
   macro avg       0.74      0.73      0.73      1448
weighted avg       0.75      0.75      0.75      1448



###### 